### Preparing the test domain

In [ ]:
# TODO: also move to dataset repo
# stdlib
import os

if not os.path.exists("ages_dataset.csv"):
    !curl -O https://openminedblob.blob.core.windows.net/csvs/ages_dataset.csv

if not os.path.exists("ages_mock_dataset.csv"):
    !curl -O https://openminedblob.blob.core.windows.net/csvs/ages_mock_dataset.csv

In [ ]:
# syft absolute
import syft as sy

SYFT_VERSION = ">=0.9.0,<0.9.1"
sy.requires(SYFT_VERSION)

# third party
import pandas as pd

# launching a test node
node = sy.orchestra.launch(name="low_domain", port=8088, dev_mode=False, reset=True)
node = sy.orchestra.launch(name="high_domain", port=8089, dev_mode=False, reset=True)

age_df = pd.read_csv("ages_dataset.csv")
age_df = age_df.dropna(how="any")

age_mock_df = pd.read_csv("ages_mock_dataset.csv")
age_mock_df = age_mock_df.dropna(how="any")

# logging in with default credentials (just for the example)
domain_low = sy.login(email="info@openmined.org", password="changethis", port=8088)
domain_high = sy.login(email="info@openmined.org", password="changethis", port=8089)

main_contributor = sy.Contributor(
    name="Jeffrey Salazar", role="Uploader", email="jsala@ailab.com"
)

low_side_asset = sy.Asset(
    name="asset_name",
    data=age_mock_df,  # real dataframe
    mock=age_mock_df,  # mock dataframe
    contributors=[main_contributor],
)

high_side_asset = sy.Asset(
    name="Age Data 2023",
    data=age_df,
    mock=age_df,
    contributors=[main_contributor],
)

low_side_dataset = sy.Dataset(
    name="Dataset name",
    description="**Dataset description**",
    asset_list=[low_side_asset],
    contributors=[main_contributor],
)

high_side_dataset = sy.Dataset(
    name="Dataset name",
    description="**Dataset description**",
    asset_list=[high_side_asset],
    contributors=[main_contributor],
)

domain_low.upload_dataset(low_side_dataset)
domain_high.upload_dataset(high_side_dataset)

domain_low.settings.allow_guest_signup(enable=True)

In [ ]:
# stdlib

# User register onto the discoverable low-side node
domain = sy.login_as_guest(port=8088).register(
    email="scientist@test.com",
    password="123",
    name="Curious Scientist",
    password_verify="123",
    institution="",
)

domain = sy.login(port=8088, email="scientist@test.com", password="123")
# User inspects the data

# User proposes a research project
project = sy.Project(
    name="Old Age Analysis",
    description="A data science project investigating if the dataset is skewed or biased towards older people.",
    members=[domain],
)

# User proposes the research code that implements the project


@sy.syft_function_single_use(data=domain.datasets[0].assets[0])
def count_over50(data):
    return data["Age of death"] > 50


@sy.syft_function(
    input_policy=sy.ExactMatch(data=domain.datasets[0].assets[0]),
    output_policy=sy.SingleExecutionExactOutput(),
)
def between_60_and_70(data):
    return data[data["Age of death"] >= 60 & data["Age of death"] <= 70]


# User submits the code requests alongside the project and awaits approval
project.create_code_request(count_over50, domain)
project.create_code_request(between_60_and_70, domain)

In [ ]:
# @leriomaggio I think this was changed to .send
# project.start() # deprecated
project.send()

In [ ]:
low_side_domain = sy.login(port=8088, email="info@openmined.org", password="changethis")
low_side_domain

In [ ]:
low_side_domain.projects

In [ ]:
project = low_side_domain.projects[0]

project.requests

In [ ]:
request = project.requests[0]
request

In [ ]:
request.code

In [ ]:
# @leriomaggio renamed to run()
# request.code.unsafe_function(data=low_side_domain.datasets[0].assets[0].mock)

In [ ]:
request.code.run(data=low_side_domain.datasets[0].assets[0].mock)

In [ ]:
high_side_domain = sy.login(
    port=8089, email="info@openmined.org", password="changethis"
)

In [ ]:
# renamed to run

In [ ]:
# result = request.code.unsafe_function(data=high_side_domain.datasets[0].assets[0].data)
# result

In [ ]:
result = request.code.run(data=high_side_domain.datasets[0].assets[0].data)
result

In [ ]:
# request.accept_by_depositing_result(result)

In [ ]:
# # @leriomaggio
# cant run
# # submitted_project.requests[0].accept_by_depositing_result(result)

# cant accept low request
# # submitted_project.requests[0].approve_with_client(low_client)

In [ ]:
project.requests

In [ ]:
request.deny(
    reason="The release of aggregate statistics for column X is not permitted according to our data release policies"
)

In [ ]:
project.requests